In [28]:
import pandas as pd
import numpy as np
import os
import time
import pickle
import re
from tqdm import tqdm
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

In [37]:
pd.set_option('display.max_columns', None)

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [45]:
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from IPython.core.pylabtools import figsize

## Login

In [3]:
ctg_url = 'https://cleaningtheglass.com'

In [4]:
ctg_login_url = 'https://cleaningtheglass.memberful.com/auth/sign_in'

In [5]:
def login(driver, cred, ctg_login_url=ctg_login_url):
    driver.get(ctg_login_url)

    # Get username & password
    with open(cred, 'r') as f:
        user_password = f.readlines()[0].replace('\n', '').split(', ')

    # Login
    userfield = driver.find_element_by_id('session_email')
    userfield.send_keys(user_password[0])
    driver.find_element_by_xpath("//button[@class='btn-main btn-expanded']").click()

    time.sleep(3)

    passwordfield = driver.find_element_by_id('session_password')
    passwordfield.send_keys(user_password[1])
    driver.find_element_by_xpath("//button[@data-session-form-target='passwordButton']").click()

In [6]:
service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)



====== WebDriver manager ======
Current google-chrome version is 110.0.5481
Get LATEST chromedriver version for 110.0.5481 google-chrome
Driver [/Users/xulian/.wdm/drivers/chromedriver/mac64/110.0.5481.77/chromedriver] found in cache


In [7]:
cred = 'ctg_cred.txt'
login(driver, cred)

## Basic Function

In [8]:
def fetch(driver):
    """
    Use BeautifulSoup to get the text of a web page.
    """
    d = driver.page_source
    comm = re.compile("<!--|-->")
    soup = BeautifulSoup(comm.sub("", d), "html.parser")
    return soup

## Scrape On-Off Data

In [26]:
def scrape_player_on_off_table(season, driver=driver):
    on_off_url = (
        f'{ctg_url}/stats/players?'
        'stat_category=onoff_efficiency&'
        f'season={season}'
    )
    driver.get(on_off_url)
    
    time.sleep(10)
    soup = fetch(driver)
    
    table = soup.find_all('table', {'id': 'players-table'})[0]
    
    # Build dataframe
    cols = ['Player', 'Age', 'Team', 'Pos', 'Min',
            'Diff_rank', 'Diff', 'Exp_W']
    for side in ['off', 'def']:
        cols += ['']
        for factor in ['Pts/Poss', 'eFG%', 'TOV%', 'OREB%', 'FTr']:
            cols = cols + [f'{side}_{factor}_rank', f'{side}_{factor}']

    data = [[td.text for td in row.find_all('td')]
            for row in table.find_all('tbody')[0].find_all('tr')]
    df = pd.DataFrame(data, columns=cols).drop('', axis=1)
    for col in cols:
        if 'rank' in col:
            df[col] = df[col].apply(lambda x: x.strip('\n').strip(' ').strip('\n'))
    df['season'] = season
    
    return df
    

In [25]:
seasons = list(range(2017, 2022))

In [29]:
season_dfs = []
for season in tqdm(seasons):
    season_dfs.append(scrape_player_on_off_table(season))

100%|█████████████████████████████████████████████| 5/5 [01:58<00:00, 23.62s/it]


In [32]:
df = pd.concat(season_dfs)
df.head()

,Player,Age,Team,Pos,Min,Diff_rank,Diff,Exp_W,off_Pts/Poss_rank,off_Pts/Poss,...,def_Pts/Poss,def_eFG%_rank,def_eFG%,def_TOV%_rank,def_TOV%,def_OREB%_rank,def_OREB%,def_FTr_rank,def_FTr,season
0,Alex Abrines,24.3,OKC,Wing,1066,46,-0.7,-2,67,+1.4,...,+2.1,49,-0.0%,24,-1.0%,9,+3.0%,18,+2.8,2017
1,Quincy Acy,27.1,BKN,Big,1287,67,+2.3,+6,70,+1.7,...,-0.7,50,-0.1%,66,+0.6%,63,-0.7%,25,+2.1,2017
2,Steven Adams,24.3,OKC,Big,2465,88,+7.6,+19,91,+6.3,...,-1.4,41,+0.5%,34,-0.6%,84,-2.8%,67,-1.0,2017
3,Arron Afflalo,32.0,ORL,Wing,618,28,-4.1,-9,30,-2.7,...,+1.4,86,-2.4%,9,-1.9%,10,+2.9%,58,-0.3,2017
4,LaMarcus Aldridge,32.3,SAS,Big,2474,93,+10.4,+27,91,+6.3,...,-4.1,75,-1.5%,25,-0.9%,87,-3.1%,41,+0.8,2017


In [33]:
df.to_csv('ctg_on_off.csv', index=False)

## Overview

In [35]:
df = pd.read_csv('ctg_on_off.csv')
df.shape

(2490, 29)

In [43]:
df = df[df['Min'] >= 400]
df.shape

(1923, 29)

In [44]:
positions = df['Pos'].unique()

,Player,Age,Team,Pos,Min,Diff_rank,Diff,Exp_W,off_Pts/Poss_rank,off_Pts/Poss,off_eFG%_rank,off_eFG%,off_TOV%_rank,off_TOV%,off_OREB%_rank,off_OREB%,off_FTr_rank,off_FTr,def_Pts/Poss_rank,def_Pts/Poss,def_eFG%_rank,def_eFG%,def_TOV%_rank,def_TOV%,def_OREB%_rank,def_OREB%,def_FTr_rank,def_FTr,season
0,Alex Abrines,24.3,OKC,Wing,1066,46,-0.7,-2,67,1.4,37,-0.8%,89,-1.4%,12,-3.7%,79,2.2,32,2.1,49,-0.0%,24,-1.0%,9,+3.0%,18,2.8,2017
1,Quincy Acy,27.1,BKN,Big,1287,67,2.3,6,70,1.7,80,+1.7%,24,+0.8%,30,-1.6%,45,-0.5,54,-0.7,50,-0.1%,66,+0.6%,63,-0.7%,25,2.1,2017
2,Steven Adams,24.3,OKC,Big,2465,88,7.6,19,91,6.3,69,+0.9%,11,+1.6%,100,+11.6%,14,-3.3,62,-1.4,41,+0.5%,34,-0.6%,84,-2.8%,67,-1.0,2017
3,Arron Afflalo,32.0,ORL,Wing,618,28,-4.1,-9,30,-2.7,55,+0.1%,23,+0.8%,33,-1.2%,78,1.9,37,1.4,86,-2.4%,9,-1.9%,10,+2.9%,58,-0.3,2017
4,LaMarcus Aldridge,32.3,SAS,Big,2474,93,10.4,27,91,6.3,74,+1.3%,92,-1.7%,65,+1.1%,36,-1.2,84,-4.1,75,-1.5%,25,-0.9%,87,-3.1%,41,0.8,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2476,Keifer Sykes,27.8,IND,Point,524,53,0.1,0,23,-4.3,39,-1.1%,82,-1.2%,2,-6.4%,69,1.4,83,-4.4,57,-0.4%,85,+1.6%,12,+3.1%,79,-2.0,2021
2477,Terry Taylor,22.1,IND,Forward,669,31,-3.7,-7,63,1.1,70,+1.1%,40,+0.3%,73,+1.8%,28,-1.9,13,4.8,3,+4.7%,74,+1.1%,81,-2.5%,86,-2.7,2021
2484,Brandon Williams,21.9,POR,Point,603,35,-3.1,-2,16,-5.5,18,-2.7%,57,-0.2%,76,+2.1%,2,-6.8,70,-2.4,67,-1.0%,74,+1.1%,55,-0.4%,25,2.1,2021
2486,Olivier Sarr,22.7,OKC,Big,400,85,6.3,14,54,-0.0,91,+2.9%,9,+2.2%,86,+3.4%,13,-3.6,92,-6.3,73,-1.4%,88,+1.9%,86,-3.0%,87,-2.8,2021
